In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-o9ct6wyt
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-o9ct6wyt
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-h0elmoqm/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmphue_6gf2".


In [ ]:
%%cuda
#include <stdio.h>

#define TILE_WIDTH 10

/*matrix multiplication kernels*/

//non shared
__global__ void MatrixMul( int *Md , int *Nd , int *Pd , const int WIDTH )
{

           // calculate thread id

           unsigned int col = TILE_WIDTH * blockIdx.x + threadIdx.x ;

           unsigned int row = TILE_WIDTH*blockIdx.y + threadIdx.y ;

           Pd[row*WIDTH + col]=0;

         for (int k = 0 ; k<WIDTH ; k++ )
         {
                  Pd[row*WIDTH + col]+= Md[row * WIDTH + k ] * Nd[ k * WIDTH + col] ;
          }

}


// main routine
int main ()
{
   const int WIDTH = 25 ;      // Change the width to change input size
   float naive_gpu_elapsed_time_ms;
   int array1_h[WIDTH][WIDTH] ,array2_h[WIDTH][WIDTH],result_array_h[WIDTH][WIDTH];
  int *array1_d , *array2_d ,*result_array_d ; // device array
  int i , j ;
  //input in host array
 // printf("Enter matrix1\n");
  for ( i = 0 ; i < WIDTH ; i++ )
  {
     for (j = 0 ; j < WIDTH ; j++ )
     {

    	 array1_h[i][j]=1;
    	 array2_h[i][j]=1;
     }
  }

cudaEvent_t start, stop;

cudaEventCreate(&start);

cudaEventCreate(&stop);

  //create device array cudaMalloc ( (void **)&array_name, sizeofmatrixinbytes) ;

  cudaMalloc((void **) &array1_d , WIDTH*WIDTH*sizeof (int) ) ;

  cudaMalloc((void **) &array2_d , WIDTH*WIDTH*sizeof (int) ) ;




  //copy host array to device array; cudaMemcpy ( dest , source , WIDTH , direction )

  cudaMemcpy ( array1_d , array1_h , WIDTH*WIDTH*sizeof (int) , cudaMemcpyHostToDevice ) ;

  cudaMemcpy ( array2_d , array2_h , WIDTH*WIDTH*sizeof (int) , cudaMemcpyHostToDevice ) ;



  //allocating memory for resultant device array

  cudaMalloc((void **) &result_array_d , WIDTH*WIDTH*sizeof (int) ) ;





  //calling kernal

  dim3 dimGrid ( WIDTH/TILE_WIDTH , WIDTH/TILE_WIDTH ,1 ) ;

  dim3 dimBlock( TILE_WIDTH, TILE_WIDTH, 1 ) ;

cudaEventRecord(start, 0);

  MatrixMul <<<dimGrid,dimBlock>>> ( array1_d , array2_d ,result_array_d , WIDTH) ;

cudaEventRecord(stop, 0);

  cudaMemcpy(result_array_h , result_array_d , WIDTH*WIDTH*sizeof(int) ,cudaMemcpyDeviceToHost) ;



  printf("matrix result\n");
  for ( i = 0 ; i < WIDTH ; i++ )
  {
      for ( j = 0 ; j < WIDTH ; j++ )
     {
        printf ("%d   ",result_array_h[i][j] ) ;
     }
 printf ("\n") ;
}
   cudaEventElapsedTime(&naive_gpu_elapsed_time_ms, start, stop);
   printf("Time elapsed on naive GPU matrix multiplication of %dx%d . %dx%d : %f ms.\n\n", WIDTH, WIDTH, WIDTH,WIDTH, naive_gpu_elapsed_time_ms);
 system("pause") ;
}



sh: 1: pause: not found
matrix result
25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   0   0   0   0   0   
25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   0   0   0   0   0   
25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   0   0   0   0   0   
25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   0   0   0   0   0   
25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   0   0   0   0   0   
25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   0   0   0   0   0   
25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   0   0   0   0   0   
25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   25   0   0   0   0  